In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import json

from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


import warnings
warnings.simplefilter("ignore")

In [ ]:
mpd.__version__

In [ ]:
pd.set_option("max_columns", None)

## Loading sample AIS data 


In [ ]:
%%time
df = gpd.read_file('../data/interim/mssis-ais-records_v2.gpkg')
wgs84 = df.crs
print("Finished reading {}".format(len(df)))

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
%%time
wpi = gpd.read_file('../data/interim/nga-wpi_v1.gpkg')
wgs84 = wpi.crs
print("Finished reading {}".format(len(wpi)))

In [ ]:
wpi.head()

In [ ]:
wpi.info()

In [ ]:
wpi.columns.tolist()

In [ ]:
df.plot()

In [ ]:
wpi.plot()

## ICL TAC1 Service

In [ ]:
df = df[df.service_id == 'TAC1']

In [ ]:
df.info()

In [ ]:
df.plot()

In [ ]:
#convert wpi from gpd.GeoDataFrame to pd.DataFrame so they can be joined and result is a GeoDataFrame
ports_wpi = pd.DataFrame(wpi)

In [ ]:
ports_wpi.drop(
    columns=[
#         'portNumber',
#         'portName',
#         'regionNumber',
#         'regionName',
#         'countryCode',
#         'countryName',
#         'publicationNumber',
        'chartNumber',
        'navArea',
        'harborSize',
        'harborType',
        'globalId',
        'dnc',
#         'alternateName',
#         's23WaterBody'
        'geometry'
    ], 
    inplace=True)

In [ ]:
ports_wpi.info()

In [ ]:
ports_wpi.head()

# join ais and nga wpi

In [ ]:
# pd.merge(left=df, right=wpi, how='left', left_on=['mssis_wpi'], right_on=['portNumber'], suffixes=('_ais', '_nga_wpi')).info()

In [ ]:
# df = pd.merge(left=df, right=wpi, how='left', left_on=['mssis_wpi'], right_on=['portNumber'], suffixes=('_ais', '_nga_wpi'))

df = pd.merge(left=df, right=ports_wpi, how='left', left_on=['mssis_wpi'], right_on=['portNumber'], suffixes=('_ais', '_nga_wpi'))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.groupby(['mssis_eez', 'mssis_ao', 'mssis_wpi']).size().reset_index().rename(columns={0:'ais_count'}).sort_values('ais_count', ascending=False)

In [ ]:
df.groupby(['mssis_eez', 'mssis_ao', 'mssis_wpi', 'portName', 'countryName']).size().reset_index().rename(columns={0:'ais_count'}).sort_values('ais_count', ascending=False)

In [ ]:
df['t'] = pd.to_datetime(df['ais_time'])
df = df.set_index('t')

In [ ]:
df.info()

In [ ]:
df['ais_sog'].hist(bins=100, figsize=(15,3))

In [ ]:
df.info()

In [ ]:
print("Original size: {} rows".format(len(df)))
df = df[df.ais_sog>0]
print("Reduced to {} rows after removing 0 speed records".format(len(df)))
df['ais_sog'].hist(bins=100, figsize=(15,3))

In [ ]:
df['vessel_name'].value_counts().plot(kind='bar', figsize=(15,3))

In [ ]:
df['portName'].value_counts().plot(kind='bar', figsize=(15,3))

## create trajectories for each vessel

In [ ]:
wgs84 = df.crs

In [ ]:
%%time
MIN_LENGTH = 100 # meters
traj_collection = mpd.TrajectoryCollection(df, 'vessel_mmsi', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
traj_collection = mpd.MinTimeDeltaGeneralizer(traj_collection).generalize(tolerance=timedelta(minutes=1))

In [ ]:
# https://stackoverflow.com/questions/28999287/generate-random-colors-rgb
# import matplotlib.pyplot as plt
import random

mmsi_colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(df.vessel_mmsi.unique().size)]

In [ ]:
mmsi_list = df.vessel_mmsi.unique().tolist()

In [ ]:
mmsi_to_color = dict(zip(mmsi_list, mmsi_colors))

In [ ]:
mmsi_to_color

In [ ]:
traj_collection.plot(column='vessel_mmsi', column_to_color=mmsi_to_color, linewidth=1, capstyle='round')

In [ ]:
traj_collection.hvplot(title='vessel_mmsi', line_width=2)

In [ ]:
print(traj_collection.trajectories[0])

In [ ]:
len(traj_collection.trajectories)

In [ ]:
traj_collection.trajectories[0].df.head()

In [ ]:
independent_pursuit = traj_collection.trajectories[0]

In [ ]:
independent_pursuit.hvplot(title='Trajectory {}'.format(str(independent_pursuit.id)), height=300, line_width=5.0, c='ais_sog', cmap='Dark2') 

In [ ]:
for traj in traj_collection:
    traj.add_speed
#     print(traj.to_linestring())

In [ ]:
independent_pursuit.to_linestring()

In [ ]:
independent_pursuit.df.head()

In [ ]:
independent_pursuit.df.info()

In [ ]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=5))
# print("Extracted {} individual trips from {} continuous vessel tracks".format(len(independent_pursuit_trips), len(independent_pursuit)))

In [ ]:
len(trips)

In [ ]:
trips.hvplot(title='trips', line_width=2)

In [ ]:
independent_pursuit.df.columns.tolist()

In [ ]:
def traj_to_timestamped_geojson(trajectory_collection):
    features = []
    
    for trajectory in traj_collection.trajectories:
    
        df = trajectory.df.copy()
        df["previous_geometry"] = df["geometry"].shift()
        df["time"] = df.index
        df["previous_time"] = df["time"].shift()
        
        for _, row in df.iloc[1:].iterrows():
            coordinates = [
                [
                    row["previous_geometry"].xy[0][0],
                    row["previous_geometry"].xy[1][0]
                ],
                [
                    row["geometry"].xy[0][0],
                    row["geometry"].xy[1][0]
                ]
            ]
            times = [row["previous_time"].isoformat(), row["time"].isoformat()]
            data = row.to_dict()
            data.pop('geometry', None)
            features.append(
                {
                    "type": "Feature",
                    "geometry": {
                        "type": "LineString",
                        "coordinates": coordinates,
                    },
                    "properties": {
                        "times": times,
    #                     "style": {
    # #                         "color": mmsi_to_color["vessel_mmsi"],
    #                         "weight": 5,
    #                     },
                        "vessel_mmsi": row["vessel_mmsi"],
#                          'ais_sog',
#                          'ais_heading',
#                          'mssis_wpi',
#                          'mssis_eez',
#                          'mssis_ao',
                         
                         "vessel_name": row["vessel_name"],
#                          'vessel_call_sign',
#                          'vessel_build_year',
#                          'vessel_gross_tonnage',
#                          'vessel_type',
#                          'vessel_flag_country',
                         "vessel_capacity_teu": row["vessel_capacity_teu"],
#                          'vessel_capacity_vehicle_units',
#                          'vessel_stern_ramp_capacity_tons',
                         "carrier_id_fk": row["carrier_id_fk"],
                         "service_id": row["service_id"],
#                          'geometry',
                         "nga_wpi": row["portNumber"],
                         "nga_wpi_port_name":row["portName"],
#                          'regionNumber',
#                          'regionName',
#                          'countryCode',
#                          'countryName',
#                          'publicationNumber',
#                          'alternateName',
#                          's23WaterBody']
                    },
                }
            )
    return features

In [ ]:
# traj_to_timestamped_geojson(independent_pursuit)

In [ ]:
# traj_features = list()

# for t in traj_collection:
#     features.append(traj_to_timestamped_geojson(traj))
    
# geojson = {
#   "type": "FeatureCollection",
#   "features": features
# }

In [ ]:
# traj_features = []

# for t in traj_collection.trajectories:
#     traj_features.append(
#         #         {
#         #             "type": "FeatureCollection",
#         #             "features": traj_to_timestamped_geojson(traj)
#         #         }
#         traj_to_timestamped_geojson(traj_features, t)
#     )

In [ ]:
geojson = {
  "type": "FeatureCollection",
  "features": traj_to_timestamped_geojson(traj_collection)
}

In [ ]:
# for f in features:
#     geojson = {
#       "type": "FeatureCollection",
#       "features": features
#     }
    
#     out.append(geojson)



In [ ]:
out_path = '../data/interim/timestamped-trajectory-icl-tac1'

In [ ]:
with open(out_path + '.json', 'w') as json_file:
    json.dump(geojson, json_file, indent=2)

In [ ]:
with open(out_path + '.geojson', 'w') as json_file:
    json.dump(geojson, json_file)

In [ ]:
# xy = t.get_start_location()
# m = folium.Map(location=[xy.y, xy.x], zoom_start=14)

# TimestampedGeoJson(
#     {
#         "type": "FeatureCollection",
#         "features": features,
#     },
#     period="PT1S",
#     add_last_point=True,
#     transition_time=10
# ).add_to(m)

# m

In [ ]:
# https://jingwen-z.github.io/how-to-draw-a-variety-of-maps-with-folium-in-python/
i=0
for mmsi in mmsi_list:
    color_dict[mmsi] = {
        'MMSI': mmsi,
        'color': color[i],
        'feature': folium.FeatureGroup(
                      name=mmsi,
                      show=False
                  )
    }
    i+=1

In [ ]:
for idx, row in geo_df.iterrows():
    
    if row['MMSI'] == 211256150:
        mmsi = row['MMSI']
        c = color_dict[mmsi]['color']

        folium.CircleMarker(
#             location=[row.geometry.y, row.geometry.x],
            radius='1',
            #color=c,
            #fill_color=c,
            color='red',
            fill_color='red',
            fill=True
        ).add_to(color_dict[mmsi]['feature'])

In [ ]:
for k, v in color_dict.items():
    v['feature'].add_to(ais_map)

folium.LayerControl(
    name='MMSI',
    collapsed=True
).add_to(ais_map)

In [ ]:
ais_map

In [ ]:
ais_map['t'] = pd.to_datetime(df['TimeOfFix'], unit='s')
df = df.set_index('t')

In [ ]:
%%time
# df = read_file('../data/ais.gpkg')
wgs84 = df.crs
print("Finished reading {}".format(len(df)))

Let's see what the data looks like:

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.TimeOfFix.min()

In [ ]:
df.TimeOfFix.max()

In [ ]:
df.plot()

To convert the DataFrame to Trajectories we need to create a temporal index:

In [ ]:
df['t'] = pd.to_datetime(df['TimeOfFix'], unit='s')
df = df.set_index('t')

If we look at the data distributions, we can see that there are a lot of records with speed over ground (SOG) values of zero in this dataframe:

In [ ]:
df['SOG'].hist(bins=100, figsize=(15,3))

Let's get rid of these rows with zero SOG:

In [ ]:
print("Original size: {} rows".format(len(df)))
df = df[df.SOG>0]
print("Reduced to {} rows after removing 0 speed records".format(len(df)))
df['SOG'].hist(bins=100, figsize=(15,3))

Let's see what kind of ships we have in our dataset:

In [ ]:
# df['ShipType'].value_counts().plot(kind='bar', figsize=(15,3))

Finally, let's create trajectories:

In [ ]:
%%time
MIN_LENGTH = 100 # meters
traj_collection = mpd.TrajectoryCollection(df, 'MMSI', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
traj_collection = mpd.MinTimeDeltaGeneralizer(traj_collection).generalize(tolerance=timedelta(minutes=1))

## Plotting trajectories

Let's give the most common ship types distinct colors. The remaining ones will be just grey:

In [ ]:
# shiptype_to_color = {'Passenger': 'blue', 'HSC': 'green', 'Tanker': 'red', 'Cargo': 'orange'}
# traj_collection.plot(column='ShipType', column_to_color=shiptype_to_color, linewidth=1, capstyle='round')
traj_collection.plot(column='MMSI', linewidth=1, capstyle='round')

In [ ]:
# passenger = traj_collection.filter('ShipType', 'Passenger')
# passenger.hvplot(title='Passenger ferries', line_width=2)
# vessels = traj_collection
traj_collection.hvplot(title='All', line_width=2)

## Visualizing trajectory properties

We can also plot individual trajectories to better visualize their properties, such as the changes in NavStatus:

In [ ]:
my_traj = traj_collection.trajectories[0]
my_traj.df.head()

In [ ]:
my_traj.df.tail()

In [ ]:
my_traj.hvplot(title='Trajectory {}'.format(str(my_traj.id)), height=300, line_width=5.0, c='NavStatus', cmap='Dark2') 

## Finding ships passing under Älvsborgsbron bridge
We can find ships passing under the bridge based on trajectory intersections with the bridge area.

In [ ]:
area_of_interest = Polygon([(11.89935, 57.69270), (11.90161, 57.68902), (11.90334, 57.68967), (11.90104, 57.69354), (11.89935, 57.69270)])

In [ ]:
intersecting = traj_collection.get_intersecting(area_of_interest)
print("Found {} intersections".format(len(intersecting)))

In [ ]:
bridge_traj = intersecting.trajectories[0]
bridge_traj.hvplot(title='Trajectory {}'.format(str(bridge_traj.id)), height=300, line_width=5.0, c='NavStatus', cmap='Dark2') 

In [ ]:
bridge_traj.df.head()

## Identifying trip origins and destinations

Since AIS records with a speed over ground (SOG) value of zero have been removed from the dataset, we can use the `split_by_observation_gap()` function to split the continuous observations into individual trips:

In [ ]:
trips = mpd.ObservationGapSplitter(passenger).split(gap=timedelta(minutes=5))
print("Extracted {} individual trips from {} continuous vessel tracks".format(len(trips), len(passenger)))

Let's plot the resulting trips!

In [ ]:
trips.hvplot(title='Passenger ferry trips', line_width=2)

Compared to plotting the original continuous observations, this visualization is much cleaner because there are no artifacts at the border of the area of interest. 

Next, let's get the trip origins:

In [ ]:
origins = trips.get_start_locations()
origins.hvplot(title='Trip origins by ship type', c='Name', geo=True, tiles='OSM')

In our data sample, trip origins can be:
- When a ship departs its anchoring location and the speed changes from 0 to >0
- When a ship trajectory first enters the observation area

In [ ]:
origins.hvplot(title='Origins by speed', c='SOG', geo=True, tiles='OSM')

## Finding ships that depart from Sjöfartsverket (Maritime Administration)

In [ ]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=5))
area_of_interest = Polygon([(11.86815, 57.68273), (11.86992, 57.68047), (11.87419, 57.68140), (11.87288, 57.68348), (11.86815, 57.68273)])

We can identify vessels that start their trip within a given area of interest by intersecting trip starting locations with our area of interest:

In [ ]:
departures = []
for traj in trips:
    if traj.get_start_location().intersects(area_of_interest) and traj.get_length() > 100:
        departures.append(traj)
print("Found {} departures".format(len(departures)))

In [ ]:
departures[1].hvplot(title='Trajectory {}'.format(departures[1].id), line_width=5, c='Name', cmap='Dark2') 

Let's see what kind of ships depart from here:

In [ ]:
for traj in departures:
    print("{} vessel '{}' departed at {}".format(traj.df['ShipType'].iloc[0], traj.df['Name'].iloc[0], traj.get_start_time()))

Of course, the same works for arrivals:

In [ ]:
arrivals = []
for traj in trips:
    if traj.get_end_location().intersects(area_of_interest) and traj.get_length() > 100:
        arrivals.append(traj)
print("Found {} arrivals".format(len(arrivals)))

for traj in arrivals:
    print("{} vessel '{}' arrived at {}".format(traj.df['ShipType'].iloc[0], traj.df['Name'].iloc[0], traj.get_end_time()))

## Clustering origins

To run this section, you need to have the scikit-learn package installed. 

In [ ]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [ ]:
origins = trips.get_start_locations()
origins['lat'] = origins.geometry.y
origins['lon'] = origins.geometry.x
matrix = origins[['lat','lon']].values

In [ ]:
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian

In [ ]:
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(matrix))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([matrix[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
origins['cluster'] = cluster_labels

In [ ]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return Point(tuple(centermost_point)[1], tuple(centermost_point)[0])
centermost_points = clusters.map(get_centermost_point)

In [ ]:
origins.hvplot(title='Clustered origins', c='cluster', geo=True, tiles='OSM', cmap='glasbey_dark')

In [ ]:
origins_by_cluster = pd.DataFrame(origins).groupby(['cluster'])
summary = origins_by_cluster['ShipType'].unique().to_frame(name='types')
summary['n'] = origins_by_cluster.size()
summary['symbol_size'] = summary['n']*10 # for visualization purposes
summary['sog'] = origins_by_cluster['SOG'].mean()
summary['geometry'] = centermost_points
summary = summary[summary['n']>1].sort_values(by='n', ascending=False)
summary.head()

In [ ]:
cluster_of_interest_id = 28
origins[origins['cluster']==cluster_of_interest_id].hvplot(
    title='Cluster {}'.format(cluster_of_interest_id), c='ShipType', geo=True, tiles='OSM', height=500)

In [ ]:
( trips.hvplot(title='Origin clusters by speed', color='gray', line_width=1) *
  GeoDataFrame(summary, crs=wgs84).hvplot(c='sog', size='symbol_size', geo=True,  cmap='RdYlGn')
)

## Continue exploring MovingPandas


1. [Ship data analysis](2-ship-data.ipynb)
2. [Bird migration analysis](1-bird-migration.ipynb)
3. [Horse collar data exploration](3-horse-collar.ipynb)
4. [Stop hotspot detection](4-stop-hotspots.ipynb)